<a href="https://colab.research.google.com/github/ianara01/Park-Sangjin/blob/master/simulation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time

In [2]:
src = cv2.imread('mot_color70.jpg') # 전체 이미지
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 입력 이미지
# 반환되는 결과 이미지의 정밀도
# X 방향으로 미분할 차수를 설정합니다.
# Y 방향으로 미분할 차수를 설정합니다.
# 소벨 마스크의 커널 크기(1, 3, 5, 7 등의 홀수 값을 사용) 최대 31
sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, 3)
print('소벨 마스크 결과')
cv2_imshow(sobel)
print()

# 입력 이미지
# 반환되는 결과 이미지의 정밀도
# 라플라시안 필터의 크기(커널의 값이 1일 경우, 중심값이 -4인 3 x 3 Aperture Size를 사용)
laplacian = cv2.Laplacian(gray, cv2.CV_8U, ksize=3)
print('라플라시안 필터 결과')
cv2_imshow(laplacian)
print()

# 입력 이미지
# 하위 임곗값
# 상위 임곗값
canny = cv2.Canny(src, 100, 255)
print('캐니 에지 결과')
cv2_imshow(canny)
print()

error: ignored

In [3]:
img = cv2.imread('mot_color70.jpg') # 전체 이미지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SIFT 알고리즘 적용
sift = cv2.SIFT_create()
kp,des = sift.detectAndCompute(gray, None)

# gray = cv2.drawKeypoints(gray, kp, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
gray = cv2.drawKeypoints(gray, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2_imshow(gray)

error: ignored

In [4]:
img1=cv2.imread('mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 입력 이미지로 사용
gray1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2=cv2.imread('mot_color83.jpg')			     # 전체 이미지
gray2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

sift=cv2.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1, None)
kp2,des2=sift.detectAndCompute(gray2, None)
print('특징점 개수:',len(kp1),len(kp2))

start=time.time()
flann_matcher=cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1, des2, 2)	# 최근접 2개

# 최근접 거리 비율 전략
thresh=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/
        nearest2.distance)<thresh:
        good_match.append(nearest1)
print('매칭에 걸린 시간:',time.time()-start)

TypeError: ignored

In [5]:
h1, w1, _ = img1.shape
h2, w2, _ = img2.shape

points1 = np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2 = np.float32([kp2[gm.trainIdx].pt for gm in good_match])
T, _ = cv2.findHomography(points1, points2, cv2.RANSAC) # 변환 행렬 추정 with RANSAC
box1 = np.float32([[0,0], [0,h1-1], [w1-1,h1-1], [w1-1,0]]).reshape(4,1,2)
box2 = cv2.perspectiveTransform(box1, T)

img2 = cv2.polylines(img2, [np.int32(box2)], True, (0,255,0), 8)

img_match = np.empty((max(h1,h2), w1+w2, 3), dtype=np.uint8)
cv2.drawMatches(img1, kp1, img2, kp2,good_match, img_match,
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2_imshow(img_match)

NameError: ignored